ARCPY LAB #1: 
Rachel Roberts
Febuary 3, 2016
Worked together with Will Chen. Discussed the joining of tables, the SQL statement, how to 'select by attributes', how to output files, store the output files, and the use of cursors and lists.

In [3]:
#import Arcpy
import sys 
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy

arcpy.env.workspace = r"U:\ARCPY_LAB1\DATA\saep_bg10"

fc = r"saep_bg10.shp"
ft = r"WashingtonFIPS.dbf"

In [2]:
# Join Table to Featureclass.  
# To have all the appropriate information in a single place, the .dbf file with the table containing 
# county names and FIPS codes needed to be joined to the shapefile. To do so, the 
# arcpy.JoinField_management function was used. This part of the assignment took approximaetly 45 minutes. 
# This however, does not take in consideration the amount of time it takes to join the tables, which was 
# approximately 10 minutes. 

# Set the local parameters
in_data = "saep_bg10.shp"
in_field = "COUNTYFP10"
join_table = "WashingtonFIPS.dbf"
join_field = "FIPSCounty"

# Join the table to the feature class by the FIPSCounty/CountyFP10 fields. 
arcpy.JoinField_management (in_data, in_field, join_table, join_field)  

<Result 'U:\\ARCPY_LAB1\\DATA\\saep_bg10\\saep_bg10.shp'>

In [4]:
# Import moduel to convert shapefiles to geoJSON files. 
from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W\\apps\\msys\\bin;C:\\OSGeo4W\\apps\\Python27\\Scripts"

In [6]:
# Select all blocks with the same FIPS code. 
# This section of the code produces shapefiles with county names, containing the appropriate census tracts based on 
# FIPs codes. To do so, a cursor was created to run through all 39 counties in Washington Shate. A loop was then
# created to produce the output files, and an SQL query based on the arcpy.Select_analysis function. The last section of
# code converts the shapefiles to geoJSON files. This part of the assignment took approximately 6 hours. 

county_list = []
FIPS_list = []

with arcpy.da.SearchCursor (ft, ["CountyName", "FIPSCounty"]) as cursor: #database file is used to only run through 39 rows.
    for row in cursor: 
        county_list.append(row[0])
        FIPS_list.append(row[1])
    del row
    del cursor 

# Create a layer based on FIPS codes and county names.
# Set output paths.

for n in range (0, len(FIPS_list)):
    fips_num = FIPS_list[n]
    county_name = county_list[n]
    shapefile_output = county_name + ".shp"
    shapefile_destination = "U:\\ARCPY_LAB1\\DATA\\saep_bg10\\county_shapefiles\\" + shapefile_output 
    geoJSON_output = county_name + ".geojson"
    geoJSON_destination = "U:\\ARCPY_LAB1\\DATA\\saep_bg10\\geoJSON_files\\" + geoJSON_output
    
# SQL Query 
    SQL_Query = '"FIPSCounty" = \'' + fips_num + '\'' 
    arcpy.Select_analysis(fc, shapefile_destination, SQL_Query)
    
# Convert files from shapefiles to geoJSON file.
    call(['C:\\OSGeo4W\\bin\\ogr2ogr', '-f','GeoJSON','-t_srs',
          'WGS84', '-s_srs','EPSG:2927', geoJSON_destination, shapefile_destination])

In [27]:
# Ranking population according to county.
# Print out a ranked list, in descending order, of the ten largest total populations in 2013 
# of counties in Washington, according to this dataset. To do so the environment workspace needed to 
# be reset to the folder containing the county shapefiles. A loop and cursor were then created to based
# on the sum function. This will produce the total population per county. The last section of the code
# sorts the values, and is then reversed to produce the values in decending order. The last line of 
# code prints out the first ten values. This section of code took approximately 1.5 hours.


arcpy.env.workspace = r"U:\ARCPY_LAB1\DATA\saep_bg10\county_shapefiles"
countyshp_list = arcpy.ListFeatureClasses()
total_pop2013 = []

for shapefile in countyshp_list:
    pop2013 = []
    with arcpy.da.SearchCursor (shapefile, ["POP2013"]) as cursor:
        for row in cursor:
            pop2013.append(row[0])
            
        county_pop = math.fsum(pop2013) 
        total_pop2013.append(county_pop)
        
total_pop2013.sort()
total_pop2013.sort (reverse=True)
print total_pop2013 [:10]

[1981900.0, 814500.0, 730500.0, 480000.0, 435500.0, 260100.0, 254000.0, 247250.0, 205800.0, 183400.0]
